In [2]:
# ORM：Object-Relational Mapping，把关系数据库的表结构映射到对象上

# # 导入:
from sqlalchemy import Column, String, create_engine,Integer
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

# 创建对象的基类:
Base = declarative_base()

# 定义User对象:
class User(Base):
    # 表的名字:
    __tablename__ = 'user'
    #  表的结构:
    ## 字段类型要声明长度
    id = Column(Integer, primary_key=True)
    name = Column(String(20))
    fullname = Column(String(50))
    nickname = Column(String(50))
    def __repr__(self):
        return "<User(name='%s', id='%s')>" % (self.name, self.id)
# 初始化数据库连接:
engine = create_engine(r'sqlite:///text.sqlite',echo=True)
# 创建DBSession类型,使用时要实例化
DBSession = sessionmaker(bind=engine)
# 创建数据库表
Base.metadata.create_all(engine)
#若修改了表结构，可能需要手动去修改schema，



2020-11-21 19:31:06,951 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-11-21 19:31:06,953 INFO sqlalchemy.engine.base.Engine ()
2020-11-21 19:31:06,958 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-11-21 19:31:06,960 INFO sqlalchemy.engine.base.Engine ()
2020-11-21 19:31:06,966 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("user")
2020-11-21 19:31:06,969 INFO sqlalchemy.engine.base.Engine ()


- 插入数据

In [3]:
#1 先实例化对象
add_=User(name='lin',nickname='werido',fullname='hongji')
#2 插入数据库,此时未写入数据库,这里记得把session实例化
session=DBSession()
session.add(add_)
#3 写入数据库
session.commit()

2020-11-21 19:31:13,554 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-11-21 19:31:13,559 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, fullname, nickname) VALUES (?, ?, ?)
2020-11-21 19:31:13,560 INFO sqlalchemy.engine.base.Engine ('lin', 'hongji', 'werido')
2020-11-21 19:31:13,571 INFO sqlalchemy.engine.base.Engine COMMIT


- 插入多条，补充说明


In [9]:
session.add_all([
  User(name='lin1',nickname='werido',fullname='hongji'),
  User(name='lin2',nickname='werido',fullname='hongji'),
  User(name='lin3',nickname='werido',fullname='hongji')
])
##  添加后未写入数据库，但是可以查询得到,commit才会写入数据库
lin=session.query(User).filter_by(name='lin').first()
lin_all=session.query(User).filter_by(name='lin1').all()
print(lin,lin_all)

## 展示事物中存在的未commit的记录
session.new

2020-11-21 19:36:17,426 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, fullname, nickname) VALUES (?, ?, ?)
2020-11-21 19:36:17,432 INFO sqlalchemy.engine.base.Engine ('lin1', 'hongji', 'werido')
2020-11-21 19:36:17,435 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, fullname, nickname) VALUES (?, ?, ?)
2020-11-21 19:36:17,447 INFO sqlalchemy.engine.base.Engine ('lin2', 'hongji', 'werido')
2020-11-21 19:36:17,455 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, fullname, nickname) VALUES (?, ?, ?)
2020-11-21 19:36:17,461 INFO sqlalchemy.engine.base.Engine ('lin3', 'hongji', 'werido')
2020-11-21 19:36:17,472 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.fullname AS user_fullname, user.nickname AS user_nickname 
FROM user 
WHERE user.name = ?
 LIMIT ? OFFSET ?
2020-11-21 19:36:17,478 INFO sqlalchemy.engine.base.Engine ('lin', 1, 0)
2020-11-21 19:36:17,482 INFO sqlalchemy.engine.base.Engine SELECT user.id AS use

IdentitySet([])

- 修改数据

In [12]:
lin.name='lin_mend'
session.dirty
# 已经修改过，会记录在session.dirty 里面提交后会清除记录
session.commit()

2020-11-21 22:26:05,681 INFO sqlalchemy.engine.base.Engine COMMIT
